# Composable Indices Demo

In [3]:
from gpt_index import GPTTreeIndex, GPTKeywordTableIndex, SimpleDirectoryReader

ImportError: cannot import name 'get_chunk_size_given_prompt' from 'gpt_index.indices.utils' (/Users/jerryliu/Programming/gpt_index/gpt_index/indices/utils.py)

#### Load Datasets

Load both the NYC Wikipedia page as well as Paul Graham's "What I Worked On" essay

In [2]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('../test_wiki/data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [ ]:
# load NYC dataset
nyc_documents = SimpleDirectoryReader('../test_wiki/data/nyc_text.txt').load_data()

In [1]:
# load PG's essay
essay_documents = SimpleDirectoryReader('../paul_graham_essay/data/paul_graham_essay.txt').load_data()

NameError: name 'SimpleDirectoryREader' is not defined

### Building the document indices
Build a tree index for the NYC wiki page and PG essay

In [ ]:
# If you haven't built the index yet, you can rebuilt here
nyc_index = GPTTreeIndex(nyc_documents)
nyc_index.save_to_disk('../test_wiki/index.json')

In [ ]:
essay_index = GPTTreeIndex(essay_documents)
essay_index.save_to_disk('../paul_graham_essay/index.json')

### Loading the indices
Build a tree indices for the NYC wiki page and PG essay

In [ ]:
# try loading
nyc_index = GPTTreeIndex.load_from_disk('../test_wiki/index.json')
essay_index = GPTTreeIndex.load_from_disk('../paul_graham_essay/index.json')

### Build Keyword Table Index on top of Index! 